# Soss implementation of the radon model

Import libraries

In [1]:
using JSON
using Soss, Distributions, NamedTupleTools
using Random
using ArviZ

Check ArviZ python version is up to date, if necessary, update to latest python ArviZ

In [2]:
ArviZ.arviz_version()

v"0.10.0"

In [3]:
# using Conda
# Conda.add("arviz"; channel="conda-forge")
# using Pkg
# Pkg.build("ArviZ")

In [4]:
radon_data = JSON.parsefile("radon.json");
radon_data["county"] = convert(Array{Int64,1}, radon_data["county"]) .+ 1;
radon_data["county_name"] = convert(Array{String,1}, radon_data["county_name"]);
radon_data["x"] = convert(Array{Int64,1}, radon_data["x"]);
radon_data["u"] = convert(Array{Float64,1}, radon_data["u"]);
radon_data["y"] = convert(Array{Float64,1}, radon_data["y"]);

In [5]:
nchains = 4
ndraws = 500

500

In [6]:
# Bayesian linear regression.
mod = @model (
    floor_idx, 
    county_idx, 
    uranium, 
    J,
) begin
    # Hyperpriors:
    g ~ Normal(0, 10) |> iid(2)
    sigma_a ~ Exponential(1)
    
    # Varying intercepts uranium model:
    a = g[1] .+ g[2] .* uranium
    za_county ~ Normal(0, 1) |> iid(J)
    a_county = a .+ za_county .* sigma_a
    # Common slope:
    b ~ Normal(0, 1)
    
    # Expected value per county:
    theta = a_county[county_idx] + b .* floor_idx
    # Model error:
    sigma ~ Exponential(1)
    
    y ~ For(eachindex(floor_idx)) do i
            Normal(theta[i], sigma)
        end
end

@model (floor_idx, county_idx, uranium, J) begin
        sigma_a ~ Exponential(1)
        sigma ~ Exponential(1)
        g ~ Normal(0, 10) |> iid(2)
        b ~ Normal(0, 1)
        a = g[1] .+ g[2] .* uranium
        za_county ~ Normal(0, 1) |> iid(J)
        a_county = a .+ za_county .* sigma_a
        theta = a_county[county_idx] + b .* floor_idx
        y ~ For(eachindex(floor_idx)) do i
                Normal(theta[i], sigma)
            end
    end


In [7]:
constant_data = (
    floor_idx = radon_data["x"], 
    county_idx = radon_data["county"],
    uranium = radon_data["u"],
    J = radon_data["J"],
)
param_mod = mod(; constant_data...);

In [8]:
prior_priorpred = [
    map(1:nchains*ndraws) do _
        draw = rand(param_mod)
        return delete(draw, keys(constant_data))
    end
];

In [9]:
post = map(1:nchains) do _
    dynamicHMC(param_mod, (y = radon_data["y"],), ndraws)
end;

In [10]:
pred = predictive(mod, :sigma_a, :sigma, :b, :g, :za_county)
post_postpred = map(post) do post_draws
    map(post_draws) do post_draw
        pred_draw = rand(pred(; constant_data..., post_draw...))
        pred_draw = delete(pred_draw, keys(constant_data))
        return merge(pred_draw, post_draw)
    end
end;

In [11]:
coords = Dict(
    "level" => ["basement", "floor"],
    "obs_id" => 1:radon_data["N"],
    "county" => radon_data["county_name"],
    "g_coef" => ["intercept", "slope"],
)
dims = Dict(
    "g" => ["g_coef"],
    "za_county" => ["county"],
    "y" => ["obs_id"],
    "floor_idx" => ["obs_id"],
    "county_idx" => ["obs_id"],
    "theta" => ["obs_id"],
    "uranium" => ["county"],
    "a" => ["county"],
    "a_county" => ["county"],
    
)
idata = from_namedtuple(
    post_postpred;
    posterior_predictive = [:y],
    prior = prior_priorpred,
    prior_predictive = [:y],
    observed_data = (y = radon_data["y"],),
    constant_data = constant_data,
    coords = coords,
    dims = dims,
    library = Soss,
);

In [12]:
using PyCall
@pyimport numpy

In [13]:
post = idata.posterior
sigma = numpy.array(post.sigma.broadcast_like(post.theta))

log_likelihood = pdf.(
    Normal.(post.theta[:values], sigma), 
    reshape(radon_data["y"],1,1,radon_data["N"])
);

In [16]:
idata = concat(idata, from_dict(log_likelihood=Dict("y" => log_likelihood), coords=coords, dims=dims))

InferenceData with groups:
	> posterior
	> posterior_predictive
	> log_likelihood
	> prior
	> prior_predictive
	> observed_data
	> constant_data

In [18]:
idata.to_netcdf("soss.nc")

"soss.nc"